# Construção da ABT
_ABT - Analytical Base Table_

---

## Sumário

1. **Importação da bibliotecas**
2. **Criação e iniciação de uma sessão PySpark**
3. **Criação dos datasets a partir da leitura dos arquivos \*.csv e \*.json**
    - 3.1. Arquivos CSV
    - 3.2. Arquivos JSON
4. **Análise dos dados**
5. **Construção da ABT**

<br>

---

<br>

## 1. Importação de bibliotecas

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, expr, count, lit
import json
import csv

## 2. Criação e iniciação de uma sessão PySpark

In [2]:
appName = 'ABT - Transações de clientes'

# O objeto SparkSession configurado com configurações específicas
spark = SparkSession.builder \
    .appName(appName) \
    .config('spark.driver.memory', '8g') \
    .config('spark.executor.memory', '8g') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()
    
spark.sparkContext.setLogLevel('ERROR')

spark

## 3. Criação dos datasets a partir da leitura dos arquivos *.csv e *.json

In [3]:
# Caminho dos arquivos *.csv e *.json
caminho = 'dados/'

### 3.1. Arquivos CSV

- cards_data.csv

In [4]:
# Dados de cartão de crédito
df_cards = spark.read.csv(caminho + 'cards_data.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_cards.printSchema()

root
 |-- id: integer (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- card_brand: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- card_number: long (nullable = true)
 |-- expires: string (nullable = true)
 |-- cvv: integer (nullable = true)
 |-- has_chip: string (nullable = true)
 |-- num_cards_issued: integer (nullable = true)
 |-- credit_limit: string (nullable = true)
 |-- acct_open_date: string (nullable = true)
 |-- year_pin_last_changed: integer (nullable = true)
 |-- card_on_dark_web: string (nullable = true)



- transactions_data.csv

In [5]:
# Dados de transações
df_transactions = spark.read.csv(caminho + 'transactions_data.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_transactions.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- card_id: integer (nullable = true)
 |-- amount: string (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_id: integer (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- zip: double (nullable = true)
 |-- mcc: integer (nullable = true)
 |-- errors: string (nullable = true)



- users_data.csv

In [6]:
# Dados de clientes
df_clients = spark.read.csv(caminho + 'users_data.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_clients.printSchema()

root
 |-- id: integer (nullable = true)
 |-- current_age: integer (nullable = true)
 |-- retirement_age: integer (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- birth_month: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- per_capita_income: string (nullable = true)
 |-- yearly_income: string (nullable = true)
 |-- total_debt: string (nullable = true)
 |-- credit_score: integer (nullable = true)
 |-- num_credit_cards: integer (nullable = true)



### 3.2. Arquivos JSON

In [7]:
def process_json_to_csv(json_path, csv_path, list_columns, one_column=None):
    '''
    Processa um arquivo JSON e converte em CSV.

    :param json_path: path + filename.json
        Caminho do arquivo JSON de entrada.
    :param csv_path: path + filename.csv
        Caminho do arquivo CSV de saída.
    :param list_columns: list
        Lista com os nomes das colunas do CSV.
    :param one_column: string (default=None)
        Nome da chave no JSON para acessar dados aninhados (opcional).
    '''
    # Carregar o arquivo JSON
    with open(json_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        
    if not data:
        print('Arquivo JSON está vazio.')
        return
    
    # Processar dados aninhados, se necessário
    if one_column:
        data = data.get(one_column, {})
        if not data:
            print(f'A chave \'{one_column}\' não foi encontrada ou está vazia no JSON.')
            return
        print(f'Dados extraídos da chave \'{one_column}\'.')
    else:
        print('Processando JSON sem dados aninhados.')

    # Escrever o CSV
    with open(csv_path, 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(list_columns)  # Escrevendo o cabeçalho
        
        for key, value in data.items():
            writer.writerow([key, value])  # Escrevendo as linhas
    
    print(f'Arquivo CSV gerado com sucesso em {csv_path}.')

- mcc_codes.json

In [8]:
# Processar mcc_codes.json
json_path = caminho + 'mcc_codes.json'
csv_path = caminho + 'mcc_codes.csv'
list_columns = ['code', 'description']
process_json_to_csv(json_path, csv_path, list_columns)

Processando JSON sem dados aninhados.
Arquivo CSV gerado com sucesso em dados/mcc_codes.csv.


In [9]:
# Dados de códigos de categoria de mercadorias
df_mcc = spark.read.csv(caminho + 'mcc_codes.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_mcc.printSchema()

root
 |-- code: integer (nullable = true)
 |-- description: string (nullable = true)



- train_fraud_labels.json

In [10]:
# Processar train_fraud_labels.json
json_path = caminho + 'train_fraud_labels.json'
csv_path = caminho + 'train_fraud_labels.csv'
list_columns = ['transaction_id', 'is_fraud']
one_column = 'target'
process_json_to_csv(json_path, csv_path, list_columns, one_column)

Dados extraídos da chave 'target'.
Arquivo CSV gerado com sucesso em dados/train_fraud_labels.csv.


In [11]:
# Dados de rótulos de fraude
df_train_fraud = spark.read.csv(caminho + 'train_fraud_labels.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_train_fraud.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- is_fraud: string (nullable = true)



## 4. Análise dos dados

- Volumetria dos dataframes

In [12]:
def dataframe_volumetry(df, df_name):
    '''
    Exibe a quantidade de linhas e colunas de um dataframe.
    
    :param df: DataFrame
        O dataframe a ser analisado.
    :param df_name: string
        Nome do dataframe a ser analisado.
    '''
    print(f'Volumetria do \033[3m{df_name}\033[0m')
    print(f'Linhas: {df.count()}')
    print(f'Colunas: {len(df.columns)}\n')

In [13]:
dataframe_volumetry(df_cards, 'df_cards')
dataframe_volumetry(df_transactions, 'df_transactions')
dataframe_volumetry(df_clients, 'df_clients')
dataframe_volumetry(df_mcc, 'df_mcc')
dataframe_volumetry(df_train_fraud, 'df_train_fraud')

Volumetria do df_cards
Linhas: 6146
Colunas: 13

Volumetria do df_transactions
Linhas: 13305915
Colunas: 12

Volumetria do df_clients
Linhas: 2000
Colunas: 14

Volumetria do df_mcc
Linhas: 109
Colunas: 2

Volumetria do df_train_fraud
Linhas: 8914963
Colunas: 2



- Criação de Views temporárias para cada dataframe

In [14]:
df_cards.createOrReplaceTempView('tb_cards')
df_clients.createOrReplaceTempView('tb_clients')
df_mcc.createOrReplaceTempView('tb_mcc')
df_train_fraud.createOrReplaceTempView('tb_train_fraud')
df_transactions.createOrReplaceTempView('tb_transactions')

- Visualizando os dados da tb_cards

In [15]:
df_cards.show(10, truncate=False)

+----+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|id  |client_id|card_brand|card_type      |card_number     |expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_web|
+----+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|4524|825      |Visa      |Debit          |4344676511950444|12/2022|623|YES     |2               |$24295      |09/2002       |2008                 |No              |
|2731|825      |Visa      |Debit          |4956965974959986|12/2020|393|YES     |2               |$21968      |04/2014       |2014                 |No              |
|3701|825      |Visa      |Debit          |4582313478255491|02/2024|719|YES     |2               |$46414      |07/2003       |2004                 |No              |
|42 

- Visualizando os dados da tb_clients

In [16]:
df_clients.show(10, truncate=False)

+----+-----------+--------------+----------+-----------+------+------------------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|id  |current_age|retirement_age|birth_year|birth_month|gender|address                 |latitude|longitude|per_capita_income|yearly_income|total_debt|credit_score|num_credit_cards|
+----+-----------+--------------+----------+-----------+------+------------------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|825 |53         |66            |1966      |11         |Female|462 Rose Lane           |34.15   |-117.76  |$29278           |$59696       |$127613   |787         |5               |
|1746|53         |68            |1966      |12         |Female|3606 Federal Boulevard  |40.76   |-73.74   |$37891           |$77254       |$191349   |701         |5               |
|1718|81         |67            |1938      |11         |Female|766 Third Drive         |34.02  

- Visualizando os dados da tb_mcc

In [17]:
df_mcc.show(10, truncate=False)

+----+------------------------------------------+
|code|description                               |
+----+------------------------------------------+
|5812|Eating Places and Restaurants             |
|5541|Service Stations                          |
|7996|Amusement Parks, Carnivals, Circuses      |
|5411|Grocery Stores, Supermarkets              |
|4784|Tolls and Bridge Fees                     |
|4900|Utilities - Electric, Gas, Water, Sanitary|
|5942|Book Stores                               |
|5814|Fast Food Restaurants                     |
|4829|Money Transfer                            |
|5311|Department Stores                         |
+----+------------------------------------------+
only showing top 10 rows



- Visualizando os dados da tb_train_fraud

In [18]:
df_train_fraud.show(10, truncate=False)

+--------------+--------+
|transaction_id|is_fraud|
+--------------+--------+
|10649266      |No      |
|23410063      |No      |
|9316588       |No      |
|12478022      |No      |
|9558530       |No      |
|12532830      |No      |
|19526714      |No      |
|9906964       |No      |
|13224888      |No      |
|13749094      |No      |
+--------------+--------+
only showing top 10 rows



- Visualizando os dados da tb_transactions

In [19]:
df_transactions.show(10, truncate=False)

+-------+-------------------+---------+-------+-------+------------------+-----------+-------------+--------------+-------+----+------+
|id     |date               |client_id|card_id|amount |use_chip          |merchant_id|merchant_city|merchant_state|zip    |mcc |errors|
+-------+-------------------+---------+-------+-------+------------------+-----------+-------------+--------------+-------+----+------+
|7475327|2010-01-01 00:01:00|1556     |2972   |$-77.00|Swipe Transaction |59935      |Beulah       |ND            |58523.0|5499|NULL  |
|7475328|2010-01-01 00:02:00|561      |4575   |$14.57 |Swipe Transaction |67570      |Bettendorf   |IA            |52722.0|5311|NULL  |
|7475329|2010-01-01 00:02:00|1129     |102    |$80.00 |Swipe Transaction |27092      |Vista        |CA            |92084.0|4829|NULL  |
|7475331|2010-01-01 00:05:00|430      |2860   |$200.00|Swipe Transaction |27092      |Crown Point  |IN            |46307.0|4829|NULL  |
|7475332|2010-01-01 00:06:00|848      |3915   |$

## 5. Construção da ABT

- Alterando o nome das colunas de identificação dos dataframes

In [20]:
df_cards = df_cards.withColumnRenamed('id', 'id_card')
df_cards = df_cards.withColumnRenamed('client_id', 'client_id_card')
df_clients = df_clients.withColumnRenamed('id', 'id_client')


- Executando o _join_ entre os dataframes

In [21]:
abt_01 = df_transactions \
    .join(df_cards, df_transactions['card_id'] == df_cards['id_card'], how='inner') \
    .join(df_clients, df_transactions['client_id'] == df_clients['id_client'], how='inner') \
    .join(df_mcc, df_transactions['mcc'] == df_mcc['code'], how='inner') \
    .join(df_train_fraud, df_transactions['id'] == df_train_fraud['transaction_id'], how='left')      
    

In [22]:
abt_01.show(20, truncate=False)

+-------+-------------------+---------+-------+-------+------------------+-----------+-------------+--------------+-------+----+------+--------------+--------+-------+--------------+----------+---------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+---------+-----------+--------------+----------+-----------+------+----------------------------+--------+---------+-----------------+-------------+----------+------------+----------------+----+------------------------------------------+
|id     |date               |client_id|card_id|amount |use_chip          |merchant_id|merchant_city|merchant_state|zip    |mcc |errors|transaction_id|is_fraud|id_card|client_id_card|card_brand|card_type|card_number     |expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_web|id_client|current_age|retirement_age|birth_year|birth_month|gender|address                     |latitude|longitud

- Criação de View temporária para abt

In [23]:
abt_01.createOrReplaceTempView('tb_abt_01')

### 5.1. Análise de dados da ABT

- Panorama geral dos dados

In [51]:
query_01 = spark.sql('''
    SELECT
        YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS ANO,
        COUNT(id) AS QTDE_TRANSACOES,
        COUNT(DISTINCT client_id) AS QTDE_CLIENTES,
        SUM(CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2))) AS TOTAL_TRANSACOES,
        COUNT(DISTINCT card_id) AS QTDE_CARTOES,
        COUNT(DISTINCT merchant_id) AS QTDE_COMERCIANTES,
        SUM(CASE WHEN is_fraud != 'No' THEN 1 ELSE 0 END) AS QTDE_FRAUDES
    FROM
        tb_abt_01
    GROUP BY
        ANO
    ORDER BY
        ANO                  
''').show(20, truncate=False)


+----+---------------+-------------+----------------+------------+-----------------+------------+
|ANO |QTDE_TRANSACOES|QTDE_CLIENTES|TOTAL_TRANSACOES|QTDE_CARTOES|QTDE_COMERCIANTES|QTDE_FRAUDES|
+----+---------------+-------------+----------------+------------+-----------------+------------+
|2010|1240880        |1137         |54232556.12     |2896        |27901            |2573        |
|2011|1290770        |1167         |55778904.96     |3137        |28153            |37          |
|2012|1321672        |1177         |56832410.86     |3245        |28710            |923         |
|2013|1352808        |1190         |58284939.62     |3339        |29280            |1337        |
|2014|1365537        |1195         |58617820.51     |3418        |28987            |664         |
|2015|1388065        |1204         |59514007.43     |3473        |29616            |2189        |
|2016|1392117        |1210         |59844028.90     |3497        |29600            |2448        |
|2017|1399308       

- Análise das transações por cartão

In [25]:
query_02 = spark.sql('''
    SELECT 
        has_chip AS CONTEM_CHIP,
        use_chip AS TIPO_USO_CARTAO, 
        COUNT(*) AS QTDE_TRANSACOES,
        SUM(CASE WHEN is_fraud != 'No' THEN 1 ELSE 0 END) AS QTDE_FRAUDES
    FROM 
        tb_abt_01
    GROUP BY 
        use_chip, has_chip
    ORDER BY 
        QTDE_TRANSACOES DESC
''').show(20, truncate=False)


+-----------+------------------+---------------+------------+
|CONTEM_CHIP|TIPO_USO_CARTAO   |QTDE_TRANSACOES|QTDE_FRAUDES|
+-----------+------------------+---------------+------------+
|YES        |Swipe Transaction |5774805        |801         |
|YES        |Chip Transaction  |4780818        |3176        |
|YES        |Online Transaction|1419172        |8155        |
|NO         |Swipe Transaction |1192380        |576         |
|NO         |Online Transaction|138740         |624         |
+-----------+------------------+---------------+------------+



- Análise das transações e valores fraudulentos

In [50]:
query_03 = spark.sql('''
    SELECT 
        YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS ANO,
        COUNT(id) AS QTDE_FRAUDES, 
        AVG(CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2))) AS AVG_VALORES_FRAUDES,
        SUM(CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2))) AS SUM_VALORES_FRAUDES,
        MAX(CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2))) AS MAX_VALORES_FRAUDES,
        MIN(CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2))) AS MIN_VALORES_FRAUDES
    FROM
        tb_abt_01
    WHERE 
        is_fraud != 'No'
    GROUP BY
        ANO
    ORDER BY
        ANO
''').show(20, truncate=False)

+----+------------+-------------------+-------------------+-------------------+-------------------+
|ANO |QTDE_FRAUDES|AVG_VALORES_FRAUDES|SUM_VALORES_FRAUDES|MAX_VALORES_FRAUDES|MIN_VALORES_FRAUDES|
+----+------------+-------------------+-------------------+-------------------+-------------------+
|2010|2573        |119.680074         |307936.83          |4978.45            |-497.00            |
|2011|37          |128.254054         |4745.40            |540.30             |-388.00            |
|2012|923         |79.074171          |72985.46           |1073.75            |-492.00            |
|2013|1337        |114.708123         |153364.76          |2263.61            |-493.00            |
|2014|664         |112.793373         |74894.80           |2251.32            |-492.00            |
|2015|2189        |127.794966         |279743.18          |2930.44            |-497.00            |
|2016|2448        |128.202745         |313840.32          |2505.58            |-500.00            |


- Análise da distribuição das transações por faixa etária

In [49]:
query_03 = spark.sql('''
SELECT
    CASE
        WHEN current_age BETWEEN 0 AND 10 THEN '0-10'
        WHEN current_age BETWEEN 11 AND 20 THEN '11-20'
        WHEN current_age BETWEEN 21 AND 30 THEN '21-30'
        WHEN current_age BETWEEN 31 AND 40 THEN '31-40'
        WHEN current_age BETWEEN 41 AND 50 THEN '41-50'
        WHEN current_age BETWEEN 51 AND 60 THEN '51-60'
        WHEN current_age BETWEEN 61 AND 70 THEN '61-70'
        WHEN current_age BETWEEN 71 AND 80 THEN '71-80'
        WHEN current_age BETWEEN 81 AND 90 THEN '81-90'
        ELSE '90+'
    END AS age_range,
    COUNT(*) AS QTDE_TRANSACOES,
    SUM(CASE WHEN is_fraud != 'No' THEN 1 ELSE 0 END) AS QTDE_FRAUDES
FROM tb_abt_01
GROUP BY age_range
ORDER BY age_range
''').show(20, truncate=False)
                    

+---------+---------------+------------+
|age_range|QTDE_TRANSACOES|QTDE_FRAUDES|
+---------+---------------+------------+
|21-30    |428671         |318         |
|31-40    |2377562        |2041        |
|41-50    |3396784        |3234        |
|51-60    |2943032        |2880        |
|61-70    |2105384        |2479        |
|71-80    |970658         |1136        |
|81-90    |864655         |1077        |
|90+      |219169         |167         |
+---------+---------------+------------+



In [25]:
import sys

try:
    sys.exit()
except SystemExit:
    print('Encerrando o script, mas mantendo o ambiente interativo ativo.')


Encerrando o script, mas mantendo o ambiente interativo ativo.
